## Deliverable 2. Create a Customer Travel Destinations Map.

In [2]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [12]:
# 1. Import the WeatherPy_database.csv file. 
#pd.read_csv("..\\Studyfolder\\abc.csv")
city_data_df = pd.read_csv('../Weather_Database/WeatherPy_database.csv')
city_data_df.head()

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Description
0,0,Bengkulu,ID,2022-05-22 18:30:35,-3.8004,102.2655,76.19,90,28,3.78,moderate rain
1,1,Ribeira Grande,PT,2022-05-22 18:30:36,38.5167,-28.7000,64.80,100,75,2.30,light intensity drizzle
2,2,Touros,BR,2022-05-22 18:30:36,-5.1989,-35.4608,79.30,85,24,13.62,few clouds
3,3,Ambovombe,MG,2022-05-22 18:30:36,-25.1667,46.0833,68.07,77,15,6.69,few clouds
4,4,Cape Town,ZA,2022-05-22 18:28:48,-33.9258,18.4232,60.12,83,0,5.75,clear sky


In [15]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 70
What is the maximum temperature you would like for your trip? 90


In [16]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Description
0,0,Bengkulu,ID,2022-05-22 18:30:35,-3.8004,102.2655,76.19,90,28,3.78,moderate rain
2,2,Touros,BR,2022-05-22 18:30:36,-5.1989,-35.4608,79.30,85,24,13.62,few clouds
5,5,Butaritari,KI,2022-05-22 18:30:37,3.0707,172.7902,81.34,79,59,10.80,light rain
7,7,Saint-Philippe,RE,2022-05-22 18:30:38,-21.3585,55.7679,71.19,76,38,11.23,scattered clouds
10,10,Saint George,US,2022-05-22 18:30:40,37.1041,-113.5841,83.62,10,0,5.75,clear sky
11,11,Rikitea,PF,2022-05-22 18:30:40,-23.1203,-134.9692,75.85,76,2,13.69,clear sky
15,15,Hithadhoo,MV,2022-05-22 18:30:42,-0.6000,73.0833,83.86,72,90,11.77,overcast clouds
17,17,Carnarvon,AU,2022-05-22 18:30:43,-24.8667,113.6333,70.86,57,33,18.01,scattered clouds
19,19,Hounde,BF,2022-05-22 18:30:44,11.5000,-3.5167,89.17,40,100,7.58,overcast clouds
20,20,Atuona,PF,2022-05-22 18:30:44,-9.8000,-139.0333,77.34,81,31,17.40,light rain


In [33]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID        275
City           275
Country        275
Date           275
Lat            275
Lng            275
Max Temp       275
Humidity       275
Cloudiness     275
Wind Speed     275
Description    275
dtype: int64

In [34]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
preferred_cities_df=preferred_cities_df.dropna()


In [35]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Description,Lat,Lng,Hotel Name
0,Bengkulu,ID,76.19,moderate rain,-3.8004,102.2655,
2,Touros,BR,79.30,few clouds,-5.1989,-35.4608,
5,Butaritari,KI,81.34,light rain,3.0707,172.7902,
7,Saint-Philippe,RE,71.19,scattered clouds,-21.3585,55.7679,
10,Saint George,US,83.62,clear sky,37.1041,-113.5841,
11,Rikitea,PF,75.85,clear sky,-23.1203,-134.9692,
15,Hithadhoo,MV,83.86,overcast clouds,-0.6000,73.0833,
17,Carnarvon,AU,70.86,scattered clouds,-24.8667,113.6333,
19,Hounde,BF,89.17,overcast clouds,11.5000,-3.5167,
20,Atuona,PF,77.34,light rain,-9.8000,-139.0333,


In [36]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. # Iterate through the DataFrame.
for index, row in hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]
    
    params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key,
    "location": f"{lat},{lng}"
   }

    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [37]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df=hotel_df.dropna()
clean_hotel_df.head(15)

,City,Country,Max Temp,Description,Lat,Lng,Hotel Name
0,Bengkulu,ID,76.19,moderate rain,-3.8004,102.2655,Grage Hotel Bengkulu
2,Touros,BR,79.30,few clouds,-5.1989,-35.4608,INN NEW HORIZON
5,Butaritari,KI,81.34,light rain,3.0707,172.7902,Isles Sunset Lodge
7,Saint-Philippe,RE,71.19,scattered clouds,-21.3585,55.7679,"Chambres d'hôte ""La Trinité"""
10,Saint George,US,83.62,clear sky,37.1041,-113.5841,Best Western Plus Abbey Inn
11,Rikitea,PF,75.85,clear sky,-23.1203,-134.9692,People ThankYou
15,Hithadhoo,MV,83.86,overcast clouds,-0.6000,73.0833,Scoop Guest House
17,Carnarvon,AU,70.86,scattered clouds,-24.8667,113.6333,Hospitality Carnarvon
19,Hounde,BF,89.17,overcast clouds,11.5000,-3.5167,Carrefour où ouvriers dorment
20,Atuona,PF,77.34,light rain,-9.8000,-139.0333,Villa Enata


In [38]:
# 8a. Create the output File (CSV)
# Create the output file (CSV).
output_data_file = 'WeatherPy_Vacation.csv'

# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [50]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather </dt><dd>{Description} and {Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [55]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(-5, 31.0), zoom_level=2.5)
marker_layer = gmaps.marker_layer(locations,info_box_content=hotel_info )
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))